In [48]:
pip install fast-ml

     -------------------------------------- 42.1/42.1 kB 677.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv("trainingdataset.csv")

In [3]:
data.rename(columns={"final":"selected_text","sentiments":"sentiment"},inplace=True)

In [4]:
data.describe()

,sentiment
count,6962.000000
mean,1.097529
std,0.296699
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [5]:
data.head(20)

,selected_text,sentiment
0,want master python get my complet python cours...,1
1,i m learn becaus i want better job than drive ...,1
2,thank mosh after struggl for year with complic...,1
3,introduct what you can do with python your fir...,1
4,ik he won t see but wa such clear display how ...,1
5,wa veri nice jump in from the standpoint someo...,1
6,as i wa follow along there s an import distinc...,1
7,i m liter onli like minut in make more sens th...,1
8,ye mosh teach python here too but more importa...,1
9,i use alway fail my code class but you make so...,1


In [6]:
data["sentiment"].value_counts()

1    6283
2     679
Name: sentiment, dtype: int64

In [7]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [8]:
tweets=data["selected_text"].to_numpy(dtype=str)
labels=data["sentiment"].to_numpy(dtype=str)
labels = to_categorical(labels, 3, dtype="float32")
tokens=Tokenizer(num_words=5000)
tokens.fit_on_texts(tweets)
seq=tokens.texts_to_sequences(tweets)
tweets=pad_sequences(seq,maxlen=200)
tweets


array([[   0,    0,    0, ..., 3206, 1471, 3207],
       [   0,    0,    0, ...,  391,  198,  384],
       [   0,    0,    0, ...,   65,  357,  134],
       ...,
       [   0,    0,    0, ...,    0,    0, 3106],
       [   0,    0,    0, ...,    0,    0, 2136],
       [   0,    0,    0, ...,    0,  878,   18]])

In [9]:
labels

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_testval, y_train, y_testval = train_test_split(tweets,labels, random_state=3)

In [12]:
X_val, X_test, y_val, y_test = train_test_split(X_testval,y_testval, random_state=3)

In [13]:
print(len(X_train))
print(len(X_val))
print(len(X_test))

5221
1305
436


In [14]:
# from fast_ml.model_development import train_valid_test_split
# X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(tweets,labels, 
#                                                                             train_size=0.8, valid_size=0.1, test_size=0.1)

In [15]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense,Embedding,LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [17]:
model = Sequential()
model.add(Embedding(5000, 20))
# model.add(LSTM(15,dropout=0.5,return_sequences=True))
model.add(LSTM(15,dropout=0.5))
model.add(Dense(2,activation='softmax'))
# model.add(LSTM(20, return_sequences=True, dropout=0.2))
# model.add(LSTM(20, activation="relu", dropout=0.1, return_sequences=True))
# model.add(LSTM(20, activation="tanh", dropout=0))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy',"Precision","Recall",f1_score])


In [ ]:
checkpoint1 = ModelCheckpoint("singleLSTM.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint1])

In [16]:
from tensorflow.keras.layers import Bidirectional

In [17]:
import keras.backend as K

def f1_score(y_true, y_pred):
  
    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0.0

    # How many selected items are relevant?
    precision = c1 / (c2 + K.epsilon())

    # How many relevant items are selected?
    recall = c1 / (c3 + K.epsilon())

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [20]:
model = Sequential()
model.add(Embedding(5000, 20))
model.add(Bidirectional(LSTM(20,dropout=0.5,return_sequences=True)))
model.add(Bidirectional(LSTM(20,dropout=0.5,return_sequences=True)))

model.add(Bidirectional(LSTM(20,dropout=0.5)))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy',"Precision","Recall",f1_score])

In [21]:
checkpoint1 = ModelCheckpoint("bidirectional_model.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint1])

Epoch 1/70
164/164 [==============================] - ETA: 0s - loss: 0.3239 - accuracy: 0.8966 - precision: 0.8966 - recall: 0.8966 - f1_score: 0.8961
Epoch 1: val_accuracy improved from -inf to 0.91743, saving model to bidirectional_model.hdf5
164/164 [==============================] - 35s 167ms/step - loss: 0.3239 - accuracy: 0.8966 - precision: 0.8966 - recall: 0.8966 - f1_score: 0.8961 - val_loss: 0.2626 - val_accuracy: 0.9174 - val_precision: 0.9174 - val_recall: 0.9174 - val_f1_score: 0.9170
Epoch 2/70
164/164 [==============================] - ETA: 0s - loss: 0.2804 - accuracy: 0.9021 - precision: 0.9021 - recall: 0.9021 - f1_score: 0.9016
Epoch 2: val_accuracy did not improve from 0.91743
164/164 [==============================] - 26s 156ms/step - loss: 0.2804 - accuracy: 0.9021 - precision: 0.9021 - recall: 0.9021 - f1_score: 0.9016 - val_loss: 0.2328 - val_accuracy: 0.9174 - val_precision: 0.9174 - val_recall: 0.9174 - val_f1_score: 0.9170
Epoch 3/70
164/164 [===============

164/164 [==============================] - 25s 154ms/step - loss: 0.1100 - accuracy: 0.9640 - precision: 0.9640 - recall: 0.9640 - f1_score: 0.9642 - val_loss: 0.3228 - val_accuracy: 0.9014 - val_precision: 0.9014 - val_recall: 0.9014 - val_f1_score: 0.9013
Epoch 19/70
164/164 [==============================] - ETA: 0s - loss: 0.1063 - accuracy: 0.9649 - precision: 0.9649 - recall: 0.9649 - f1_score: 0.9651
Epoch 19: val_accuracy did not improve from 0.91743
164/164 [==============================] - 25s 154ms/step - loss: 0.1063 - accuracy: 0.9649 - precision: 0.9649 - recall: 0.9649 - f1_score: 0.9651 - val_loss: 0.3148 - val_accuracy: 0.8991 - val_precision: 0.8991 - val_recall: 0.8991 - val_f1_score: 0.8991
Epoch 20/70
164/164 [==============================] - ETA: 0s - loss: 0.1034 - accuracy: 0.9634 - precision: 0.9634 - recall: 0.9634 - f1_score: 0.9636
Epoch 20: val_accuracy did not improve from 0.91743
164/164 [==============================] - 25s 155ms/step - loss: 0.1034 -

Epoch 54/70
164/164 [==============================] - ETA: 0s - loss: 0.0417 - accuracy: 0.9868 - precision: 0.9868 - recall: 0.9868 - f1_score: 0.9869
Epoch 54: val_accuracy did not improve from 0.91743
164/164 [==============================] - 25s 154ms/step - loss: 0.0417 - accuracy: 0.9868 - precision: 0.9868 - recall: 0.9868 - f1_score: 0.9869 - val_loss: 0.5098 - val_accuracy: 0.8739 - val_precision: 0.8739 - val_recall: 0.8739 - val_f1_score: 0.8719
Epoch 55/70
164/164 [==============================] - ETA: 0s - loss: 0.0441 - accuracy: 0.9858 - precision: 0.9858 - recall: 0.9858 - f1_score: 0.9859
Epoch 55: val_accuracy did not improve from 0.91743
164/164 [==============================] - 26s 156ms/step - loss: 0.0441 - accuracy: 0.9858 - precision: 0.9858 - recall: 0.9858 - f1_score: 0.9859 - val_loss: 0.5652 - val_accuracy: 0.8693 - val_precision: 0.8693 - val_recall: 0.8693 - val_f1_score: 0.8674
Epoch 56/70
164/164 [==============================] - ETA: 0s - loss: 0.0

In [18]:
from tensorflow.keras.layers import GRU

In [19]:
# model = Sequential()
# model.add(Embedding(5000, 20))
# model.add(Bidirectional(GRU(40,dropout=0.1,return_sequences=True,activation='relu')))
# model.add(Bidirectional(GRU(40,dropout=0.30000000000000004,return_sequences=True,activation='tanh')))
# model.add(Bidirectional(LSTM(50,dropout=0.4,return_sequences=True,activation='tanh')))
# model.add(Bidirectional(LSTM(20,dropout=0.1,activation='sigmoid')))
# model.add(Dense(3,activation='softmax'))
# model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'],)
model = Sequential()
model.add(Embedding(5000, 15))
model.add(Bidirectional(GRU(15,dropout=0.1,return_sequences=True,activation='relu')))
model.add(Bidirectional(GRU(15,dropout=0.1,return_sequences=True,activation='relu')))
model.add(Bidirectional(LSTM(15,dropout=0.1,return_sequences=True,activation='softmax')))
model.add(Bidirectional(LSTM(15,dropout=0.1,activation='softmax')))
model.add(Dense(3,activation='softmax'))
model.compile(optimizer='Adam',loss='categorical_crossentropy', metrics=['accuracy',"Precision","Recall",f1_score])

In [20]:
checkpoint1 = ModelCheckpoint("bidirectionalLSTMGRU_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model.fit(X_train, y_train, epochs=100,validation_data=(X_val, y_val),callbacks=[checkpoint1])

Epoch 1/100
164/164 [==============================] - ETA: 0s - loss: 0.9335 - accuracy: 0.8221 - precision: 0.9046 - recall: 0.0600 - f1_score: nan
Epoch 1: val_accuracy improved from -inf to 0.89962, saving model to bidirectionalLSTMGRU_model2.hdf5
164/164 [==============================] - 160s 824ms/step - loss: 0.9335 - accuracy: 0.8221 - precision: 0.9046 - recall: 0.0600 - f1_score: nan - val_loss: 0.7282 - val_accuracy: 0.8996 - val_precision: 0.8996 - val_recall: 0.8996 - val_f1_score: 0.8995
Epoch 2/100
164/164 [==============================] - ETA: 0s - loss: 0.4921 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9024
Epoch 2: val_accuracy did not improve from 0.89962
164/164 [==============================] - 131s 799ms/step - loss: 0.4921 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9024 - val_loss: 0.3762 - val_accuracy: 0.8996 - val_precision: 0.8996 - val_recall: 0.8996 - val_f1_score: 0.8995
Epoch 3/100
164/164 [========

164/164 [==============================] - ETA: 0s - loss: 0.3229 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9024
Epoch 18: val_accuracy did not improve from 0.89962
164/164 [==============================] - 126s 756ms/step - loss: 0.3229 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9024 - val_loss: 0.3283 - val_accuracy: 0.8996 - val_precision: 0.8996 - val_recall: 0.8996 - val_f1_score: 0.8995
Epoch 19/100
164/164 [==============================] - ETA: 0s - loss: 0.3227 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9014
Epoch 19: val_accuracy did not improve from 0.89962
164/164 [==============================] - 120s 734ms/step - loss: 0.3227 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9014 - val_loss: 0.3280 - val_accuracy: 0.8996 - val_precision: 0.8996 - val_recall: 0.8996 - val_f1_score: 0.8995
Epoch 20/100
164/164 [==============================] - ETA: 0s - loss: 0.3224 - ac

Epoch 36/100
164/164 [==============================] - ETA: 0s - loss: 0.3208 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9024
Epoch 36: val_accuracy did not improve from 0.89962
164/164 [==============================] - 120s 732ms/step - loss: 0.3208 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9024 - val_loss: 0.3265 - val_accuracy: 0.8996 - val_precision: 0.8996 - val_recall: 0.8996 - val_f1_score: 0.8995
Epoch 37/100
164/164 [==============================] - ETA: 0s - loss: 0.3208 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9024
Epoch 37: val_accuracy did not improve from 0.89962
164/164 [==============================] - 127s 766ms/step - loss: 0.3208 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9024 - val_loss: 0.3264 - val_accuracy: 0.8996 - val_precision: 0.8996 - val_recall: 0.8996 - val_f1_score: 0.8995
Epoch 38/100
164/164 [==============================] - ETA: 0s - loss

Epoch 54/100
164/164 [==============================] - ETA: 0s - loss: 0.3203 - accuracy: 0.9023 - precision: 0.9023 - recall: 0.9023 - f1_score: 0.9028
Epoch 54: val_accuracy did not improve from 0.89962
164/164 [==============================] - 128s 768ms/step - loss: 0.3203 - accuracy: 0.9023 - precision: 0.9023 - recall: 0.9023 - f1_score: 0.9028 - val_loss: 0.3263 - val_accuracy: 0.8996 - val_precision: 0.8996 - val_recall: 0.8996 - val_f1_score: 0.8995
Epoch 55/100
164/164 [==============================] - ETA: 0s - loss: 0.3203 - accuracy: 0.9023 - precision: 0.9023 - recall: 0.9023 - f1_score: 0.9028
Epoch 55: val_accuracy did not improve from 0.89962
164/164 [==============================] - 120s 733ms/step - loss: 0.3203 - accuracy: 0.9023 - precision: 0.9023 - recall: 0.9023 - f1_score: 0.9028 - val_loss: 0.3262 - val_accuracy: 0.8996 - val_precision: 0.8996 - val_recall: 0.8996 - val_f1_score: 0.8995
Epoch 56/100
164/164 [==============================] - ETA: 0s - loss

Epoch 72/100
164/164 [==============================] - ETA: 0s - loss: 0.0953 - accuracy: 0.9728 - precision: 0.9728 - recall: 0.9726 - f1_score: 0.9708
Epoch 72: val_accuracy did not improve from 0.90038
164/164 [==============================] - 120s 734ms/step - loss: 0.0953 - accuracy: 0.9728 - precision: 0.9728 - recall: 0.9726 - f1_score: 0.9708 - val_loss: 0.2972 - val_accuracy: 0.8981 - val_precision: 0.8981 - val_recall: 0.8981 - val_f1_score: 0.8980
Epoch 73/100
164/164 [==============================] - ETA: 0s - loss: 0.0909 - accuracy: 0.9751 - precision: 0.9751 - recall: 0.9751 - f1_score: 0.9752
Epoch 73: val_accuracy improved from 0.90038 to 0.90498, saving model to bidirectionalLSTMGRU_model2.hdf5
164/164 [==============================] - 128s 781ms/step - loss: 0.0909 - accuracy: 0.9751 - precision: 0.9751 - recall: 0.9751 - f1_score: 0.9752 - val_loss: 0.2898 - val_accuracy: 0.9050 - val_precision: 0.9050 - val_recall: 0.9050 - val_f1_score: 0.9048
Epoch 74/100
164

164/164 [==============================] - ETA: 0s - loss: 0.0537 - accuracy: 0.9879 - precision: 0.9879 - recall: 0.9879 - f1_score: 0.9880
Epoch 89: val_accuracy did not improve from 0.90881
164/164 [==============================] - 106s 646ms/step - loss: 0.0537 - accuracy: 0.9879 - precision: 0.9879 - recall: 0.9879 - f1_score: 0.9880 - val_loss: 0.3633 - val_accuracy: 0.9004 - val_precision: 0.9004 - val_recall: 0.9004 - val_f1_score: 0.9003
Epoch 90/100
164/164 [==============================] - ETA: 0s - loss: 0.0508 - accuracy: 0.9891 - precision: 0.9891 - recall: 0.9891 - f1_score: 0.9891
Epoch 90: val_accuracy did not improve from 0.90881
164/164 [==============================] - 112s 681ms/step - loss: 0.0508 - accuracy: 0.9891 - precision: 0.9891 - recall: 0.9891 - f1_score: 0.9891 - val_loss: 0.3494 - val_accuracy: 0.9011 - val_precision: 0.9011 - val_recall: 0.9011 - val_f1_score: 0.9010
Epoch 91/100
164/164 [==============================] - ETA: 0s - loss: 0.0592 - ac

In [21]:
model.evaluate(X_test,y_test)

14/14 [==============================] - 1s 71ms/step - loss: 0.3790 - accuracy: 0.8968 - precision: 0.8968 - recall: 0.8968 - f1_score: 0.8955


[0.37895286083221436,
 0.896789014339447,
 0.896789014339447,
 0.896789014339447,
 0.8955357670783997]

In [19]:
from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow.keras import regularizers

In [24]:

model = Sequential()
model.add(Embedding(5000, 20))
model.add(Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model.add(MaxPooling1D(20))
model.add(Bidirectional(GRU(20,dropout=0.5,return_sequences=True)))
model.add(Bidirectional(GRU(20,dropout=0.5,return_sequences=True)))
model.add(Bidirectional(LSTM(20,dropout=0.5,return_sequences=True)))
model.add(Bidirectional(LSTM(20,dropout=0.5)))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy',"Precision","Recall",f1_score])

In [25]:
checkpoint1 = ModelCheckpoint("CNNLSTM_model.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint1])

Epoch 1/70
163/164 [============================>.] - ETA: 0s - loss: 0.4730 - accuracy: 0.9011 - precision: 0.9011 - recall: 0.9011 - f1_score: 0.9011
Epoch 1: val_accuracy improved from -inf to 0.91743, saving model to CNNLSTM_model.hdf5
164/164 [==============================] - 16s 33ms/step - loss: 0.4735 - accuracy: 0.9008 - precision: 0.9008 - recall: 0.9008 - f1_score: 0.8992 - val_loss: 0.3179 - val_accuracy: 0.9174 - val_precision: 0.9174 - val_recall: 0.9174 - val_f1_score: 0.9170
Epoch 2/70
163/164 [============================>.] - ETA: 0s - loss: 0.3285 - accuracy: 0.9018 - precision: 0.9018 - recall: 0.9018 - f1_score: 0.9018
Epoch 2: val_accuracy did not improve from 0.91743
164/164 [==============================] - 3s 19ms/step - loss: 0.3283 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9024 - val_loss: 0.2878 - val_accuracy: 0.9174 - val_precision: 0.9174 - val_recall: 0.9174 - val_f1_score: 0.9170
Epoch 3/70
163/164 [========================

Epoch 19/70
163/164 [============================>.] - ETA: 0s - loss: 0.3241 - accuracy: 0.9018 - precision: 0.9018 - recall: 0.9018 - f1_score: 0.9018
Epoch 19: val_accuracy did not improve from 0.91743
164/164 [==============================] - 3s 19ms/step - loss: 0.3239 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9024 - val_loss: 0.2885 - val_accuracy: 0.9174 - val_precision: 0.9174 - val_recall: 0.9174 - val_f1_score: 0.9170
Epoch 20/70
163/164 [============================>.] - ETA: 0s - loss: 0.3237 - accuracy: 0.9022 - precision: 0.9022 - recall: 0.9022 - f1_score: 0.9022
Epoch 20: val_accuracy did not improve from 0.91743
164/164 [==============================] - 3s 19ms/step - loss: 0.3244 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9004 - val_loss: 0.2908 - val_accuracy: 0.9174 - val_precision: 0.9174 - val_recall: 0.9174 - val_f1_score: 0.9170
Epoch 21/70
163/164 [============================>.] - ETA: 0s - loss: 0.3239 

Epoch 37/70
163/164 [============================>.] - ETA: 0s - loss: 0.3243 - accuracy: 0.9020 - precision: 0.9020 - recall: 0.9020 - f1_score: 0.9020
Epoch 37: val_accuracy did not improve from 0.91743
164/164 [==============================] - 3s 19ms/step - loss: 0.3246 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9014 - val_loss: 0.2879 - val_accuracy: 0.9174 - val_precision: 0.9174 - val_recall: 0.9174 - val_f1_score: 0.9170
Epoch 38/70
163/164 [============================>.] - ETA: 0s - loss: 0.3244 - accuracy: 0.9018 - precision: 0.9018 - recall: 0.9018 - f1_score: 0.9018
Epoch 38: val_accuracy did not improve from 0.91743
164/164 [==============================] - 3s 19ms/step - loss: 0.3242 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9024 - val_loss: 0.2875 - val_accuracy: 0.9174 - val_precision: 0.9174 - val_recall: 0.9174 - val_f1_score: 0.9170
Epoch 39/70
163/164 [============================>.] - ETA: 0s - loss: 0.3245 

Epoch 55/70
163/164 [============================>.] - ETA: 0s - loss: 0.3236 - accuracy: 0.9020 - precision: 0.9020 - recall: 0.9020 - f1_score: 0.9020
Epoch 55: val_accuracy did not improve from 0.91743
164/164 [==============================] - 3s 19ms/step - loss: 0.3238 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9014 - val_loss: 0.2904 - val_accuracy: 0.9174 - val_precision: 0.9174 - val_recall: 0.9174 - val_f1_score: 0.9170
Epoch 56/70
163/164 [============================>.] - ETA: 0s - loss: 0.3237 - accuracy: 0.9020 - precision: 0.9020 - recall: 0.9020 - f1_score: 0.9020
Epoch 56: val_accuracy did not improve from 0.91743
164/164 [==============================] - 3s 19ms/step - loss: 0.3239 - accuracy: 0.9019 - precision: 0.9019 - recall: 0.9019 - f1_score: 0.9014 - val_loss: 0.2902 - val_accuracy: 0.9174 - val_precision: 0.9174 - val_recall: 0.9174 - val_f1_score: 0.9170
Epoch 57/70
163/164 [============================>.] - ETA: 0s - loss: 0.3241 

# hyperparameter tuning

In [5]:
import tensorflow as tf
import keras_tuner as kt


In [27]:
def build_RNN_model(hp):
    
    # define model
    model = Sequential()
    model.add(Embedding(5000, 20))
    model.add(Bidirectional(GRU(hp.Int('input_unit',min_value=10,max_value=50,step=10), return_sequences=True, activation=hp.Choice("in_activation", values=["tanh", "relu", "sigmoid"]), dropout=hp.Float('in_dropout',min_value=0,max_value=.5,step=0.1))))
    model.add(Bidirectional(GRU(hp.Int('layer 1',min_value=10,max_value=50,step=10), activation=hp.Choice("l1_activation", values=["tanh", "relu", "sigmoid"]), dropout=hp.Float('l1_dropout',min_value=0,max_value=.5,step=0.1), return_sequences=True)))
    model.add(Bidirectional(LSTM(hp.Int('layer 2',min_value=10,max_value=50,step=10), activation=hp.Choice("l2_activation", values=["tanh", "relu", "sigmoid"]), dropout=hp.Float('l2_dropout',min_value=0,max_value=.5,step=0.1), return_sequences=True)))
    model.add(Bidirectional(LSTM(hp.Int('layer 4',min_value=20,max_value=50,step=10), activation=hp.Choice("l4_activation", values=["tanh", "relu", "sigmoid"]), dropout=hp.Float('l4_dropout',min_value=0,max_value=.5,step=0.1))))
    # output layer
    model.add(Dense(3,activation=hp.Choice("l5_activation", values=["tanh", "relu", "sigmoid"])))
    model.compile(optimizer=Adam(learning_rate=1e-3),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [28]:

tuner = kt.Hyperband(build_RNN_model,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=100,
                     factor=3,
                     hyperband_iterations=5,
                     directory='kt_dir',
                     project_name='rnn',
                     overwrite=True)

In [29]:
tuner.search(X_train, y_train, validation_split=0.2)

Trial 1270 Complete [00h 14m 40s]
val_accuracy: 0.8625592589378357

Best val_accuracy So Far: 0.8625592589378357
Total elapsed time: 4d 04h 20m 35s


In [19]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"input: {best_hps.get('input_unit')}")
print(f"input dropout: {best_hps.get('in_dropout')}")
print(f"input activation: {best_hps.get('in_activation')}")


print(f"layer 1: {best_hps.get('layer 1')}")
print(f"layer 1 dropout: {best_hps.get('l1_dropout')}")
print(f"layer 1 activation: {best_hps.get('l1_activation')}")


print(f"layer 2: {best_hps.get('layer 2')}")
print(f"layer 2 dropout: {best_hps.get('l2_dropout')}")
print(f"layer 2 activation: {best_hps.get('l2_activation')}")

print(f"layer 4: {best_hps.get('layer 4')}")
print(f"layer 4 dropout: {best_hps.get('l4_dropout')}")
print(f"layer 4 activation: {best_hps.get('l4_activation')}")

print(f"layer 4: {best_hps.get('layer 4')}")
print(f"layer 4 dropout: {best_hps.get('l4_dropout')}")
print(f"layer 4 activation: {best_hps.get('l4_activation')}")

input: 40
input dropout: 0.1
input activation: relu
layer 1: 40
layer 1 dropout: 0.30000000000000004
layer 1 activation: tanh
layer 2: 50
layer 2 dropout: 0.4
layer 2 activation: tanh
layer 4: 20
layer 4 dropout: 0.1
layer 4 activation: sigmoid


In [ ]:
model = Sequential()
model.add(Embedding(5000, 20))
model.add(Bidirectional(GRU(40,dropout=0.1,return_sequences=True,activation='relu')))
model.add(Bidirectional(GRU(40,dropout=0.30000000000000004,return_sequences=True,activation='tanh')))
model.add(Bidirectional(LSTM(50,dropout=0.4,return_sequences=True,activation='tanh')))
model.add(Bidirectional(LSTM(20,dropout=0.1,activation='sigmoid')))
model.add(Dense(3,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'],)

# hyperparmeter  model

In [4]:
best_hps=tuner.get_best_hyperparameters()[0]
print(f"input: {best_hps.get('input_unit')}")
print(f"input dropout: {best_hps.get('in_dropout')}")
print(f"input activation: {best_hps.get('in_activation')}")


print(f"layer 1: {best_hps.get('layer 1')}")
print(f"layer 1 dropout: {best_hps.get('l1_dropout')}")
print(f"layer 1 activation: {best_hps.get('l1_activation')}")


print(f"layer 2: {best_hps.get('layer 2')}")
print(f"layer 2 dropout: {best_hps.get('l2_dropout')}")
print(f"layer 2 activation: {best_hps.get('l2_activation')}")

print(f"layer 4: {best_hps.get('layer 4')}")
print(f"layer 4 dropout: {best_hps.get('l4_dropout')}")
print(f"layer 4 activation: {best_hps.get('l4_activation')}")

print(f"l5_activation: {best_hps.get('l5_activation')}")


NameError: name 'tuner' is not defined

In [34]:
model = Sequential()
model.add(Embedding(5000, 20))
model.add(Bidirectional(GRU(40,dropout=0.1,return_sequences=True,activation='relu')))
model.add(Bidirectional(GRU(40,dropout=0.30000000000000004,return_sequences=True,activation='relu')))
model.add(Bidirectional(LSTM(50,dropout=0.4,return_sequences=True,activation='sigmoid')))
model.add(Bidirectional(LSTM(20,dropout=0.1,activation='sigmoid')))
model.add(Dense(3,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'],)

In [35]:
checkpoint1 = ModelCheckpoint("hyperpartric_model.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model.fit(X_train, y_train, epochs=100,validation_data=(X_test, y_test),callbacks=[checkpoint1])

Epoch 1/100
33/33 [==============================] - ETA: 0s - loss: 0.8255 - accuracy: 0.6550
Epoch 1: val_accuracy improved from -inf to 0.72443, saving model to hyperpartric_model.hdf5
33/33 [==============================] - 24s 404ms/step - loss: 0.8255 - accuracy: 0.6550 - val_loss: 0.7314 - val_accuracy: 0.7244
Epoch 2/100
33/33 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.7336
Epoch 2: val_accuracy did not improve from 0.72443
33/33 [==============================] - 12s 371ms/step - loss: 0.6923 - accuracy: 0.7336 - val_loss: 0.7260 - val_accuracy: 0.7244
Epoch 3/100
33/33 [==============================] - ETA: 0s - loss: 0.6903 - accuracy: 0.7336
Epoch 3: val_accuracy did not improve from 0.72443
33/33 [==============================] - 12s 372ms/step - loss: 0.6903 - accuracy: 0.7336 - val_loss: 0.7186 - val_accuracy: 0.7244
Epoch 4/100
33/33 [==============================] - ETA: 0s - loss: 0.6861 - accuracy: 0.7336
Epoch 4: val_accuracy did not

33/33 [==============================] - 13s 385ms/step - loss: 0.0795 - accuracy: 0.9820 - val_loss: 0.6907 - val_accuracy: 0.8494
Epoch 29/100
33/33 [==============================] - ETA: 0s - loss: 0.0775 - accuracy: 0.9810
Epoch 29: val_accuracy did not improve from 0.85227
33/33 [==============================] - 12s 379ms/step - loss: 0.0775 - accuracy: 0.9810 - val_loss: 0.7311 - val_accuracy: 0.8068
Epoch 30/100
33/33 [==============================] - ETA: 0s - loss: 0.0730 - accuracy: 0.9810
Epoch 30: val_accuracy did not improve from 0.85227
33/33 [==============================] - 12s 379ms/step - loss: 0.0730 - accuracy: 0.9810 - val_loss: 0.7400 - val_accuracy: 0.8409
Epoch 31/100
33/33 [==============================] - ETA: 0s - loss: 0.0740 - accuracy: 0.9820
Epoch 31: val_accuracy did not improve from 0.85227
33/33 [==============================] - 12s 372ms/step - loss: 0.0740 - accuracy: 0.9820 - val_loss: 0.7531 - val_accuracy: 0.8352
Epoch 32/100
33/33 [========

Epoch 58/100
33/33 [==============================] - ETA: 0s - loss: 0.0688 - accuracy: 0.9877
Epoch 58: val_accuracy did not improve from 0.85227
33/33 [==============================] - 13s 383ms/step - loss: 0.0688 - accuracy: 0.9877 - val_loss: 1.0182 - val_accuracy: 0.8381
Epoch 59/100
33/33 [==============================] - ETA: 0s - loss: 0.0774 - accuracy: 0.9858
Epoch 59: val_accuracy did not improve from 0.85227
33/33 [==============================] - 13s 405ms/step - loss: 0.0774 - accuracy: 0.9858 - val_loss: 0.9517 - val_accuracy: 0.8438
Epoch 60/100
33/33 [==============================] - ETA: 0s - loss: 0.0774 - accuracy: 0.9848
Epoch 60: val_accuracy did not improve from 0.85227
33/33 [==============================] - 12s 374ms/step - loss: 0.0774 - accuracy: 0.9848 - val_loss: 1.0320 - val_accuracy: 0.8239
Epoch 61/100
33/33 [==============================] - ETA: 0s - loss: 0.0467 - accuracy: 0.9905
Epoch 61: val_accuracy did not improve from 0.85227
33/33 [=====

# Testing

In [20]:
from tensorflow.keras.models import load_model
load_model("bidirectionalLSTMGRU_model2.hdf5",custom_objects={"f1_score":f1_score}).evaluate(X_test,y_test)

14/14 [==============================] - 2s 47ms/step - loss: 0.4179 - accuracy: 0.9174 - precision: 0.9174 - recall: 0.9174 - f1_score: 0.9170


[0.41791316866874695,
 0.9174311757087708,
 0.9174311757087708,
 0.9174311757087708,
 0.9169642329216003]

In [21]:
load_model("singleLSTM.hdf5",custom_objects={"f1_score":f1_score}).evaluate(X_test,y_test)

14/14 [==============================] - 1s 12ms/step - loss: 0.2707 - accuracy: 0.9174 - precision: 0.9174 - recall: 0.9174 - f1_score: 0.9170


[0.27066031098365784,
 0.9174311757087708,
 0.9174311757087708,
 0.9174311757087708,
 0.9169642329216003]

In [22]:
load_model("bidirectional_model.hdf5",custom_objects={"f1_score":f1_score}).evaluate(X_test,y_test)

14/14 [==============================] - 2s 38ms/step - loss: 0.2626 - accuracy: 0.9174 - precision: 0.9174 - recall: 0.9174 - f1_score: 0.9170


[0.26261335611343384,
 0.9174311757087708,
 0.9174311757087708,
 0.9174311757087708,
 0.9169642329216003]

In [23]:
load_model("CNNLSTM_model.hdf5",custom_objects={"f1_score":f1_score}).evaluate(X_test,y_test)

14/14 [==============================] - 2s 5ms/step - loss: 0.3179 - accuracy: 0.9174 - precision: 0.9174 - recall: 0.9174 - f1_score: 0.9170


[0.31789612770080566,
 0.9174311757087708,
 0.9174311757087708,
 0.9174311757087708,
 0.9169642329216003]

In [43]:
prediction=load_model("bidirectionalLSTMGRU_model2.hdf5",custom_objects={"f1_score":f1_score}).predict(X_test)
prediction

14/14 [==============================] - 2s 47ms/step


array([[0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.29147798, 0.708522  ],
       [0.

In [51]:
import numpy as np
y_test
np.around(y_test, decimals=0).argmax(axis=1)

array([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [52]:
import collections
collections.Counter(np.around(y_test, decimals=0).argmax(axis=1))

Counter({0: 36, 1: 400})

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.92      1.00      0.96       400

   micro avg       0.92      0.92      0.92       436
   macro avg       0.46      0.50      0.48       436
weighted avg       0.84      0.92      0.88       436
 samples avg       0.92      0.92      0.92       436



C:\Users\conec\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
